In [1]:
%load_ext autoreload
%autoreload 2
import os#
import torch
import argparse
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
from models.networks import UnetGenerator, InceptionDown, PartialConv, InceptionUp
import numpy as np
from models.modules.shift_unet import InceptionUnetGeneratorShiftTriple
from util.util import wrapper_gmask

In [2]:
from options.train_options import TrainOptions 

In [3]:
dataroot = '/mnt/hdd2/AIM/DAGM/Class4_def/' # ENTER HERE THE PATH YOU WANT TO USE AS DATAROOT
options = '--dataroot {}'.format(dataroot).split(' ')

In [4]:
def get_parser(options=None):
    parser = TrainOptions()
    parser.parse(options=options)
    return parser

In [5]:
parser = get_parser(options=options)

----------------- Options ---------------
                batchSize: 1                             
                    beta1: 0.5                           
               bottleneck: 512                           
          checkpoints_dir: /mnt/hdd2/AIM/checks          
                constrain: MSE                           
           continue_train: False                         
                 dataroot: /mnt/hdd2/AIM/DAGM/Class4_def/	[default: /mnt/hdd2/AIM/DAGM/Class5]
             dataset_mode: aligned_resized               
             display_freq: 10                            
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
display_single_pane_ncols: 0                             
          display_winsize: 256                           
              epoch_count: 1                       

In [6]:
opt = parser.opt

In [7]:
mask = wrapper_gmask(opt)

/home/tchaton/virtualenvs/labelbox/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [8]:
input_nc = 3
output_nc = 3
num_downs = 7
innerCos_list = []
shift_list = []
mask_global = mask

In [9]:
inception_unet = InceptionUnetGeneratorShiftTriple(input_nc, output_nc, num_downs, opt, innerCos_list, shift_list, mask_global)
inception_unet

512 512 64
innermost
512 512 64
512 512 64
512 512 64
512 512 64
512 512 64
256 512 64
256 512 64
128 256 32
128 256 32
64 128 16
64 128 16
64 3 16


InceptionUnetGeneratorShiftTriple(
  (model): InceptionUnetSkipConnectionBlock(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): InceptionUnetSkipConnectionBlock(
        (model): Sequential(
          (0): InceptionDown(
            (act): LeakyReLU(negative_slope=0.2, inplace)
            (conv0_1x1_0): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_0): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv0_1x1_1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (max_pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv0_1x1_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (bconv0_1x1_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [10]:
def count_params(model):
    nb_params = 0
    for param in model.parameters():
        if param is not None:
            nb_params += np.product(param.shape)
    return nb_params

In [11]:
count_params(inception_unet)

3255635

In [12]:
x_np = np.random.normal(0, 1, (1, 3, 256, 256))
x_tr = torch.FloatTensor(x_np)

In [13]:
%time output = inception_unet(x_tr)

Input torch.Size([1, 3, 256, 256])
Input torch.Size([1, 64, 128, 128])
64 128 torch.Size([1, 64, 128, 128])
64 128 torch.Size([1, 64, 128, 128])
64 128 torch.Size([1, 64, 128, 128])
64 128 torch.Size([1, 16, 128, 128])
64 128 torch.Size([1, 16, 128, 128])
64 128 torch.Size([1, 16, 128, 128])
64 128 torch.Size([1, 64, 64, 64])
64 128 torch.Size([1, 16, 128, 128])
64 128 torch.Size([1, 16, 128, 128])
Input torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 128, 32, 32])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
Input torch.Size([1, 256, 32, 32])
256 512 torch.Size([1, 256, 32, 32])
256 512 torch.Size([1, 256, 32, 32])
256 512 torch.Size([1, 256, 32, 32])
256 512 torch.Size([1, 64, 32, 32])
256 512 torch.Size([1, 64, 32, 32])
256 512 torch.Siz

/home/tchaton/virtualenvs/labelbox/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


RuntimeError: Given transposed=1, weight of size [512, 128, 1, 1], expected input[1, 1536, 4, 4] to have 512 channels, but got 1536 channels instead

In [14]:
inception = InceptionDown(128, 256)

128 256 32


In [15]:
x_np = np.random.normal(0, 1, (1, 128, 64, 64))
x_tr = torch.FloatTensor(x_np)

In [16]:
%time output = inception(x_tr)

128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 128, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 128, 32, 32])
128 256 torch.Size([1, 32, 64, 64])
128 256 torch.Size([1, 32, 64, 64])
CPU times: user 1.36 s, sys: 44 ms, total: 1.4 s
Wall time: 70.9 ms


In [17]:
inception = InceptionUp(64, 32)

64 32 16


In [18]:
%timeit output = inception(x_tr)

64 32 torch.Size([1, 128, 64, 64])


RuntimeError: Given transposed=1, weight of size [64, 8, 1, 1], expected input[1, 128, 64, 64] to have 64 channels, but got 128 channels instead

In [ ]:
output.shape